# Assignment 03: Simple Language Model

This week, we want you to create a Language Model (LM) to judge how common a sentence is.  
More specificly, you need to <u>calculate the probability of a given setnence</u> showing in an article.  

*No need to make your output exactly the same as ours. As long as it's *reasonable* (which means you can explain it), you get your points.  

## What is a Language Model?

<b>A statistical language model is a probability distribution over sequences of words. Given such a sequence, say of length $m$, it assigns a probability $P(w_1, \ldots ,w_m)$ to the whole sequence.</b>
<i>-- from <a href="https://en.wikipedia.org/wiki/Language_model">Wikipedia</a></i>  

To put it simply, Language Model calculates the probability of a word, a sequence of words, or a sentence.  
This can be useful in many NLP tasks, like machine translation, spelling checking, speech recognition, etc.  

Consider the following two sentence  

> (1) He is looking <font color="green">*to*</font> a new job.  

and 

> (2) He is looking <font color="green">*for*</font> a new job.  


We can see that there's a grammar error in sentence 1, for which the LM should generate a lower probability.  
Hence, if your LM is well-established, you are able to judge which sentence is more correct (or more common) among a set of candidate sentences.  

But how to do so?

## Implementation

### Step 1 - Ngram frequency

First of all, we need to calculate 1- and 2-gram frequency from the corpus.  
**Again**, yes. As we have said in the first class, word/phrase frequency plays an important role in NLP.

You should be familiar with this procedure now, so let's skip the boring explanation.  
<small>*Please refer to assignment 1 and 2 if you have any question.</small>  

In [33]:
import os
from collections import Counter
from string import punctuation
with open(os.path.join('data', 'big.txt'), 'r') as f:
    corpus = f.read()

<font color="red">**[ TODO ]**</font> Calculate 1- and 2-gram frequency with the given corpus.

In [34]:
def get_ngram(tokens, n=2):
    ... # [ TODO ]
    tmp = [] #set() can remove duplicate item automatically
    tmp_ngram = ""
    for i in range(len(tokens)-n+1):
        tmp_ngram = tokens[i:i+n]
        tmp.append(" ".join(tmp_ngram))
        
    tokens = tmp
    return tokens

def calculate_frequency(tokens):
    #frequency = {token:tokens.count(token) for token in tokens}
    frequency = Counter(tokens)
    return frequency

def unigram(text):
    """ [TODO] Get the 1-gram list from string
    @param text: given string
    @return: a list of token
    """
    text = text.lower()
    Regtmp = text.replace('\n', ' ')
    Regtmp = Regtmp.split(' ')
    Reg = [i.strip(punctuation) for i in Regtmp] 
    
    tmp = get_ngram(Reg, 1)
    
    return tmp

def bigram(text):
    """ [TODO] Get the 2-gram list """
    text = text.lower()
    Regtmp = text.replace('\n', ' ')
    Regtmp = Regtmp.split(' ')
    Reg = [i.strip(punctuation) for i in Regtmp] 
    
    tmp = get_ngram(Reg, 2)
    
    return tmp


uni_freq = calculate_frequency(unigram(corpus)) # [TODO] calculate the 1-gram frequency of the corpus
bi_freq = calculate_frequency(bigram(corpus)) # [TODO] calculate the 2-gram frequency of the corpus

print(bi_freq)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Step 2: Probability of a word

#### Unigram  
Probability of a word, or a *unigram*, is simple: its occurence devided by the total word count should do the trick.  
$$P(w) = \frac{Freq(w)}{N}$$  
, where $N = $ count of all words.  
But the unigram language model doesn't consider other context words, so we choose to use bigram model here.  

#### Bigram  
As to 2-gram probability, we would use the Maximum Likelihood Estimate (MLE) to calculate it.  

$$P_{mle}(w_i|w_{i-1}) = \frac{Freq(w_{i-1}, w_i)}{Freq(w_{i-1})}$$

For example, if we have a corpus with 

> can you tell me about any good cantonese restaurants  
> tell me about chez panisse  
> can you give me a listing of the kinds of food that are available  

Then the probability of *tell* preceding *me* is $P_{mle}(me|tell) = \frac{2}{2} = 1$.  
Similarly, the probability of *me* preceding *about* is $P_{mle}(about|me) = \frac{2}{3}$.  


<font color="red">**[ TODO ]**</font> Calculate the bigram probability.

In [48]:
def P_mle(word: str, pre_word: str = None):
    """ [TODO] Return the P(word|pre_word), which should be a float """
    rnd = round(bi_freq[pre_word+" "+word]/uni_freq[pre_word],20)
    return rnd

In [49]:
p1 = P_mle('book', 'read')
p2 = P_mle('books', 'read')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0; 
p2: 0.004672897196261682


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2` (expetced to be `0`).  

In [50]:
p1 = P_mle('books', 'a')
p2 = P_mle('book', 'a')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0; 
p2: 0.001006277253342278


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2` (expetced to be `0`).  

In [51]:
p1 = P_mle('have', 'he')
p2 = P_mle('has', 'he')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0006567066163191594; 
p2: 0.015350517156460351


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2`.  

### Step 3 - Sentence probability

Now we have the probability of every word and bigram. But how should we combine them and get the probability of a sentence?  

Recall the chain rule in probability: 
$$P(X_1, X_2, X_3, X_4) = P(X_1) \cdot P(X_2|X_1) \cdot P(X_3|X_1,X_2) \cdot P(X_4|X_1, X_2, X_3)$$  
And with Markove Assumption, we know that 
$$P(w_1, w_2, \dots, w_n) \approx P(w_2|w_1) \cdot P(w_3|w_2) \cdot \dots \cdot P(w_n|w_{n-1})$$

For example,
$$P(I\:want\:a\:cake) \approx P(want|I) \cdot P(a|want) \cdot P(cake|a)$$

Since we have already built a method to calculate $P(w_{i}|w_{i-1})$, we can get the probability of a sentence through this equation.  
Note that it's recommended to sum them under $log$-scale to prevent underflow, because gram probabilities are mostly some small floating points, 
$$log(p_1 \cdot p_2 \dots p_n) = log(p_1) + log(p_2) + \dots + log(p_n) $$

<font color="red">**[ TODO ]**</font> Calculate the sentence probability. (Please do so under the `log` scale to prevent underflow)  
<small>*Hint: `math`</small>

In [52]:
import math
def sentence_prob(sentence: str, P):
    """ Calculate the probability of a given sentence with given P-model.
    @param sentence: the given sentence
    @param P: the probability model to use. (like your `P_mle` above)
    @return: a float indicating the probability
    """
    words = unigram(sentence)
    ... # [TODO] calculate the probability
    prob_sum = 0.0
    for i in range(len(words)-1):
        prob_sum = prob_sum + math.log(P(words[i+1], words[i]))
    return prob_sum

In [53]:
p1 = sentence_prob('He have to take the medicine.', P=P_mle)
p2 = sentence_prob('He has to take the medicine.', P=P_mle)
print(f'p1: {p1}; \np2: {p2}')

p1: -28.481178814885453; 
p2: -26.009285995119363


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2`.  

### Looks good so far?

However, this model heavily depends on how large your dataset is.   
If you give it the word or gram not existing in the corpus, the probability of it will be $0$, which causes problems during the calculation (because you can't divide numbers by $0$).  

Try this! 

In [54]:
sentence_prob("He has to eat medicine.", P=P_mle)

ValueError: math domain error

See? Now you get the problem.  
Surely you can expand your dataset to decrease the number of unseen phrases, but it's impossible to cover all phrases in the world.  
Here the smoothing technique comes to rescue!  

### Step 4 - smoothing

A naïve solution is to set the default frequency as `1`.  
For example, if the original frequency is

> hello: 2  
> world: 1  
> empty: 0  
> null: 0  

, after defaulting the frequency to 1, it would be

> hello: 3  
> world: 2  
> empty: <font color="red">**1**</font><br/>
> null: <font color="red">**1**</font><br/>


And if the frequency is all added by 1, the probability of bigram will now be  
$$ P_{add1} = \frac{Freq(w_{i-1}, w_i)+1}{Freq(w_{i-1})+N}$$
, where $N = count\:of\:all\:tokens$ .  

This is called the **Add-1 Estimation**, or also **Laplace Smoothing**.  

<font color="red">**[ TODO ]**</font> Implement the Laplace smoothing.  

In [55]:
def P_add1(word: str, pre_word: str):
    """ [TODO] Return the P(word|pre_word) with Laplace smoothing. """
    ...
    rnd = round( (bi_freq[pre_word+" "+word]+1)/(uni_freq[pre_word]+len(uni_freq)),20)
    return rnd

In [56]:
p1 = P_add1('medicine', 'eat')
p2 = P_add1('medicine', 'take')
print(f'p1: {p1}; \np2: {p2}')

p1: 2.601795238714713e-05; 
p2: 5.129125740517529e-05


<font color="green">Expected output:</font> `p1` shouldn't throw any error, and `p1` should be **smaller** than `p2`. 

As you can see, Add-1 estimation is a really naive method and doesn't always produce good results.  
There are better smooting techiniques like <a href="https://en.wikipedia.org/wiki/Good%E2%80%93Turing_frequency_estimation">Good-Turing Estimation</a> or <a href="https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing">Kneser-Ney Smoothing</a>. Feel free to give it a try!  

## Test your LM

Seems that you've done a great job! Let's give it some tests to see if it works as expected.  

In [57]:
test_cases = [
  'He have to take the medicine.',
  'He has to eat the medicine.',
  'He has to take the medicine.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))

He have to take the medicine. -37.238229143632296
He has to eat the medicine. -41.11627334006142
He has to take the medicine. -35.5843940865373


<font color="green">Expected output:</font> The last sentence should return the highest probability. 

In [59]:
'''
test_cases = [
  'He is looking to a job.',
  'He is looking for a job.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))
'''

test_cases = [
  'He school nice and happy.',
  'He went to school yesterday and had a nice time there.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))


He school nice and happy. -38.0644883762775
He went to school yesterday and had a nice time there. -78.90666582327225


<font color="green">Expected output:</font> The second sentence should return a higher probability. 

## Think about it
Think about the following questions. **TA will ask your answers during the demo.**  

1. What would happen if you use MLE Estimation to compare two sentences with different lengthes?  
   Examples. (1) *He school nice and happy.* and (2) *He went to school yesterday and had a nice time there.*  

2. According to Q1, Do you think MLE model is fair? If so, why? If not, how could you improve this? (Just think about it; no need to actually implement it)

**Congratulation!** You've successfully built your own language model.  
You should have caught a glimpse of LM's power despite its plainness.  
Feel free to play around with your LM and improve it, like  
1. using a larger dataset, 
2. applying better smoothing technique, or 
3. considering longer gram dependency (3- to 5-gram are resonable length).  

## TA's note


Remember to <b><a href="https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit?usp=sharing">make an appoiment with TA</a> to demo/explain your implementation <u>before <font color="red">10/7 15:30</font></u></b> .  
You should also save your file as {student_id}.ipynb and submit it to <a href="https://eeclass.nthu.edu.tw/course/homework/2384">eeclass</a> .  
**Late submission is not allowed.**  